In [1]:
import os
import glob

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import time

from skimage.feature import hog
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from scipy.ndimage.measurements import label

from moviepy.editor import VideoFileClip
from IPython.display import HTML

% matplotlib inline

In [2]:
color_space = 'YCrCb' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9  # HOG orientations
pix_per_cell = 8 # HOG pixels per cell
cell_per_block = 2 # HOG cells per block
hog_channel = "ALL" # Can be 0, 1, 2, or "ALL"
spatial_size = (16, 16) # Spatial binning dimensions
hist_bins = 16    # Number of histogram bins
spatial_feat = True # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off

cells_per_step = 2 # Windows search step; it defines windows overlaping

In [3]:
def bin_spatial(img, size=(32, 32)):
    color1 = cv2.resize(img[:,:,0], size).ravel()
    color2 = cv2.resize(img[:,:,1], size).ravel()
    color3 = cv2.resize(img[:,:,2], size).ravel()
    return np.hstack((color1, color2, color3))

def color_hist(img, nbins=32):    #bins_range=(0, 256)
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

def get_hog_features(img, orient, pix_per_cell, cell_per_block, vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), block_norm='L2-Hys', 
                                  visualise=vis, transform_sqrt=False, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), block_norm='L2-Hys', 
                       visualise=vis, transform_sqrt=False, feature_vector=feature_vec)
        return features


In [4]:
def convert_color(image, color_space):
    if color_space != 'RGB':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
    else: feature_image = np.copy(image)
    
    return feature_image

# Define a function to extract features from a list of images
# Have this function call bin_spatial(), color_hist() and get_hog_features
# Used for training the model
def extract_features(imgs, 
        color_space='RGB', spatial_size=(32, 32), hist_bins=32, orient=9, pix_per_cell=8, cell_per_block=2, hog_channel=0, 
        spatial_feat=True, hist_feat=True, hog_feat=True):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        file_features = []
        # Read in each one by one
        image = mpimg.imread(file)
        
        # apply color conversion if other than 'RGB'
        feature_image = convert_color(image, color_space)  

        if spatial_feat == True:
            spatial_features = bin_spatial(feature_image, size=spatial_size)
            file_features.append(spatial_features)
        if hist_feat == True:
            # Apply color_hist()
            hist_features = color_hist(feature_image, nbins=hist_bins)
            file_features.append(hist_features)
        if hog_feat == True:
        # Call get_hog_features() with vis=False, feature_vec=True
            if hog_channel == 'ALL':
                hog_features = []
                for channel in range(feature_image.shape[2]):
                    hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                        orient, pix_per_cell, cell_per_block, 
                                        vis=False, feature_vec=True))
                hog_features = np.ravel(hog_features)        
            else:
                hog_features = get_hog_features(feature_image[:,:,hog_channel], 
                                                orient, pix_per_cell, cell_per_block, 
                                                vis=False, feature_vec=True)
            # Append the new feature vector to the features list
            file_features.append(hog_features)
        features.append(np.concatenate(file_features))
    # Return list of feature vectors
    return features

# version of the above function for a single image
# Used for testing the image data sets or videos
def single_img_features(img, 
        color_space='RGB', spatial_size=(32, 32), hist_bins=32, orient=9, pix_per_cell=8, cell_per_block=2, hog_channel=0, 
        spatial_feat=True, hist_feat=True, hog_feat=True, 
        vis=False):    
    #1) Define an empty list to receive features
    img_features = []
    hog_image = None
    #2) Apply color conversion if other than 'RGB'
    feature_image = convert_color(img, color_space)
    #3) Compute spatial features if flag is set
    if spatial_feat == True:
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        #4) Append features to list
        img_features.append(spatial_features)
    #5) Compute histogram features if flag is set
    if hist_feat == True:
        hist_features = color_hist(feature_image, nbins=hist_bins)
        #6) Append features to list
        img_features.append(hist_features)
    #7) Compute HOG features if flag is set
    if hog_feat == True:
        counter=0
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.extend(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))  
        else:
            if vis == True:
                hog_features, hog_image = get_hog_features(feature_image[:,:,hog_channel], 
                                                           orient, pix_per_cell, cell_per_block, 
                                                           vis=True, feature_vec=True)
            else:
                hog_features = get_hog_features(feature_image[:,:,hog_channel], 
                                                orient, pix_per_cell, cell_per_block, 
                                                vis=False, feature_vec=True)
        #8) Append features to list
        img_features.append(hog_features)
    
    #9) Return concatenated array of features
    #print("sif11_len depois de HOG:", len(img_features))
    if vis == True:
        return np.concatenate(img_features), hog_image
    else:
        return np.concatenate(img_features)

In [5]:
set_dir = 'examples/set/'
vehicle_dir = 'vehicles/'
non_vehicle_dir = 'non-vehicles/'

v_image_types = os.listdir(set_dir+vehicle_dir)
nv_image_types = os.listdir(set_dir+non_vehicle_dir)
cars = []
notcars = []

for imtype in v_image_types:
    cars.extend(glob.glob(set_dir+vehicle_dir+imtype+'/*'))
print("Car images count:", len(cars))
        
for imtype in nv_image_types:
    notcars.extend(glob.glob(set_dir+non_vehicle_dir+imtype+'/*'))
print("Not car images count:", len(notcars))


Car images count: 8792
Not car images count: 8968


In [7]:
t=time.time()
n_samples = 1000
random_idxs = np.random.randint(0, len(cars), n_samples)

test_cars = cars #np.array(cars)[random_idxs]   #cars#
test_notcars = notcars #np.array(notcars)[random_idxs] #notcars#

print('Feature extraction is about to start...')

car_features = extract_features(test_cars, color_space=color_space, spatial_size=spatial_size, hist_bins=hist_bins, 
                                orient=orient, pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                                hog_channel=hog_channel, spatial_feat=spatial_feat, hist_feat=hist_feat, 
                                hog_feat=hog_feat)

print('Done with car images, starting with non-car images...')

notcar_features = extract_features(test_notcars, color_space=color_space, spatial_size=spatial_size, hist_bins=hist_bins, 
                                   orient=orient, pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                                   hog_channel=hog_channel, spatial_feat=spatial_feat, hist_feat=hist_feat, 
                                   hog_feat=hog_feat)

print(time.time()-t, 'seconds to extract features from car and non-car data sets')

# Create an array stack of feature vectors
X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))

# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.1, random_state=rand_state)

#print('Using spatial binning of:',spatial,'and', histbin,'histogram bins')
print('Using: ', orient, ' orientations, ', pix_per_cell, 
      ' pixels per cell, ', cell_per_block, ' cells per block, ', 
      hist_bins, ' histogram bins, ', spatial_size, ' spatial sampling.')
print('Feature vector length:', len(X_train[0]))
# Use a linear SVC 
svc = LinearSVC()
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
print(round(time.time()-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))


Feature extraction is about to start...
Done with car images, starting with non-car images...
219.1137638092041 seconds to extract features from car and non-car data sets
Using:  9  orientations,  8  pixels per cell,  2  cells per block,  16  histogram bins,  (16, 16)  spatial sampling.
Feature vector length: 6108
25.83 Seconds to train SVC...
Test Accuracy of SVC =  0.9837


In [8]:
def find_cars_in_image(img,ystart,window_size, scale):
    
    global pix_per_cell
    global cell_per_block
    global orient
    global svc
    global X_scaler
    
    global cells_per_step
    
    img_boxes = []
    features_per_block = orient * cell_per_block**2
    window = 64
    blocks_per_window = (window // pix_per_cell) - 1
    increased_detection = 16
    
    area_of_interest = img[ystart:ystart+window_size,:,:]
    color_converted_img = cv2.cvtColor(area_of_interest, cv2.COLOR_RGB2YCrCb)
    imshape = color_converted_img.shape
    img = cv2.resize(color_converted_img, (np.int(imshape[1]/scale), np.int(imshape[0]/scale)))
    
    hog_ch1 = get_hog_features(img[:,:,0], orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog_ch2 = get_hog_features(img[:,:,1], orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog_ch3 = get_hog_features(img[:,:,2], orient, pix_per_cell, cell_per_block, feature_vec=False)
    
    Xsteps = (img.shape[1] - window) // pix_per_cell // cells_per_step
    Ysteps = (img.shape[0] - window) // pix_per_cell // cells_per_step
    
    for x_var in range(Xsteps+1):
        for y_var in range(Ysteps+1):
            
            ypos = y_var*cells_per_step
            xpos = x_var*cells_per_step
            
            xleft = xpos*pix_per_cell
            ytop = ypos*pix_per_cell
            
            subimg = img[ytop:ytop+window, xleft:xleft+window]
            
            spatial_features = bin_spatial(subimg, size=spatial_size)
            
            hist_features = color_hist(subimg, nbins=hist_bins)
            
            hog_feat_ch1 = hog_ch1[ypos:ypos+blocks_per_window, xpos:xpos+blocks_per_window].ravel()
            hog_feat_ch2 = hog_ch2[ypos:ypos+blocks_per_window, xpos:xpos+blocks_per_window].ravel()
            hog_feat_ch3 = hog_ch3[ypos:ypos+blocks_per_window, xpos:xpos+blocks_per_window].ravel()
            hog_features = np.hstack((hog_feat_ch1, hog_feat_ch2, hog_feat_ch3))
            
            test_features = X_scaler.transform(np.hstack((spatial_features, hist_features, hog_features)).reshape(1,-1))
            test_prediction = svc.predict(test_features)
            
            # The following line can be used to display all the windows used to search cars
            #all_boxes.append(((xleft*scale, ytop*scale+ystart),(xleft*scale+window*scale,ytop*scale+window*scale+ystart)))
            
            if test_prediction == 1:
                xbox_left = np.int(xleft*scale)
                ytop_draw = np.int(ytop*scale)
                win_draw = np.int(window*scale)
                img_boxes.append((xbox_left, ystart + ytop_draw, xbox_left + win_draw, ystart + ytop_draw + win_draw))
               
    return img_boxes

In [9]:
def apply_threshold(heatmap, threshold):
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap

def draw_labeled_bboxes(img, labels):
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        bbox_center = ((np.min(nonzerox)+np.max(nonzerox))//2,(np.min(nonzeroy)+np.max(nonzeroy))//2)
        
        # Draw the box on the image
        cv2.rectangle(img, bbox[0], bbox[1], (255,0,0), 6)
        cv2.circle(img, bbox_center, 5, (0,255,0), thickness=1, lineType=8, shift=0) 
    # Return the image
    return img

In [10]:
# Define a class to receive the characteristics of each vehicle detection
# Objects defined as "Vehicles" will be where multiple overlapping detections exist in the heatmap
class Detected_window():
    def __init__(self, center, size, first, last, move):
        self.center = center          # Detection center
        self.size = size              # Detection size
        self.first_detection = first  # first detection frame
        self.last_detection = last    # last detection frame
        self.move = move              # movement from last detection

In [11]:
def process_image(frame):
    
    global frame_count
    global detection_log
    
    compare_length = 25
    threshold_level = 1
    tolerance = -0.1 # tolerance to find center of current object inside object from the previous frame
    
    updated_frame = np.copy(frame)
    
    heatmap = np.zeros_like(frame[:,:,0])
    final_heat = np.zeros_like(frame[:,:,0])
    frame = frame.astype(np.float32)/255
    
    img_boxes = []
    windows_params = []
    count = 0
    
    ystart = 400
    window_size = 160
    scale = 2.5
    windows_params.append((ystart, window_size, scale))
    
    ystart = 404
    window_size = 96
    scale = 1.5
    windows_params.append((ystart, window_size, scale))
    
    #ystart = 406
    #window_size = 64
    #scale = 1
    #windows_params.append((ystart, window_size, scale))
    
    # search cars in frame and store positive detections
    for i in range(len(windows_params)):
        img_boxes_tmp = find_cars_in_image(frame, windows_params[i][0], windows_params[i][1], windows_params[i][2])
        if len(img_boxes_tmp) > 0:
            img_boxes.append(img_boxes_tmp)
        
    # prepare images to be displayed
    if len(img_boxes) > 0:
        for j in range(len(img_boxes)):
            for i in range(len(img_boxes[j])):
                heatmap[img_boxes[j][i][1] : img_boxes[j][i][3], img_boxes[j][i][0] : img_boxes[j][i][2]] += 1
                cv2.rectangle(updated_frame, (img_boxes[j][i][0], img_boxes[j][i][1]), (img_boxes[j][i][2], img_boxes[j][i][3]),(255,255,0),1,4)
    
     # identify detections groups
    labels_before = label(heatmap)
    
    # identify detections groups after threshold
    #thresholded_heatmap = apply_threshold(heatmap, threshold_level)
    thresholded_heatmap = apply_threshold(heatmap, np.amax(heatmap)//2.5)
    labels_after = label(thresholded_heatmap)
    
    # Iterate through all detections
    for detection in range(1, labels_after[1]+1):
        nonzero = (labels_after[0] == detection).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        Xcenter = (np.min(nonzerox)+np.max(nonzerox))//2
        Ycenter = (np.min(nonzeroy)+np.max(nonzeroy))//2
        
        # add detected windows group to detection array or update previous detection:
        # - center of group: x -> [0][0] y -> [0][1]
        # - size of group: x -> [1][0] y -> [1][1]
        # - first frame where visible: [2]
        # - last frame where visible: [3]
        # Find pixels with each car_number label value
        
        match_found = 0
        if frame_count > 0 and len(detection_log) > 0:
            # check matching existing detections
            i = len(detection_log) - 1 # -1 because first detection will be element zero from the list
            
            filtered_list = [i for i in detection_log if i.last_detection > frame_count - compare_length]
            
            #while frame_count - detection_log[i].last_detection <= compare_length:
            for detected_box in filtered_list:
                if (detected_box.center[0] - detected_box.size[0] // 2 - detected_box.size[0] * tolerance < Xcenter and 
                   detected_box.center[0] + detected_box.size[0] // 2 + detected_box.size[0] * tolerance > Xcenter and
                   detected_box.center[1] - detected_box.size[1] // 2 - detected_box.size[1] * tolerance < Ycenter and 
                   detected_box.center[1] + detected_box.size[1] // 2 + detected_box.size[1] * tolerance > Ycenter):
                    detected_box.last_detection = frame_count
                    detected_box.center = (Xcenter,Ycenter)
                    detected_box.size = (np.max(nonzerox)-np.min(nonzerox), np.max(nonzeroy)-np.min(nonzeroy))
                    detected_box.move = (Xcenter - detected_box.center[0] + detected_box.move[0], Ycenter - detected_box.center[1] + detected_box.move[1])
                    match_found = 1
                    print("match found")
                    final_heat[np.min(nonzeroy):np.max(nonzeroy),np.min(nonzerox):np.max(nonzerox)] += 1
                
        if match_found == 0:
            current_detection = Detected_window((Xcenter, Ycenter),(np.max(nonzerox)-np.min(nonzerox),np.max(nonzeroy)-np.min(nonzeroy)),frame_count,frame_count,(0,0))
            detection_log.append(current_detection)
            
    labels_final = label(final_heat)
    
    updated_frame = draw_labeled_bboxes(updated_frame, labels_final)
    frame_count += 1
    
    return updated_frame

In [12]:
frame_count = 0
detection_log = []

output_file = './output_images/output_project_video.mp4' 

#clip = VideoFileClip("test_video.mp4").subclip(0,1)
clip = VideoFileClip("project_video.mp4")#.subclip(12,16)

output_clip = clip.fl_image(process_image)

%time output_clip.write_videofile(output_file, audio=False)

[MoviePy] >>>> Building video ./output_images/output_project_video.mp4
[MoviePy] Writing video ./output_images/output_project_video.mp4


  0%|          | 1/1261 [00:00<07:45,  2.71it/s]

match found


  0%|          | 4/1261 [00:01<07:47,  2.69it/s]

match found


...

100%|█████████▉| 1259/1261 [07:55<00:00,  2.60it/s]

match found
match found
match found
match found
match found
match found
match found
match found
match found
match found
match found
match found
match found


100%|█████████▉| 1260/1261 [07:56<00:00,  2.59it/s]

match found
match found
match found
match found
match found
match found
match found
match found
match found
match found
match found
match found
match found


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./output_images/output_project_video.mp4 

CPU times: user 7min 48s, sys: 13.7 s, total: 8min 1s
Wall time: 7min 56s


In [13]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output_file))